## Examples of usage

In [ ]:
import numpy as np
import random
import pandas as pd
import meliora.meliora.val_tests as vt

In [ ]:
# Import data
test_data_main = pd.read_csv('pd_test_data_main.csv')
test_data_add = pd.read_csv('pd_test_data_period_2.csv')

In [ ]:
# Binomial test
vt.binomial_test(test_data_main, "ratings", "default_flag", "predicted_pd")

In [ ]:
# Jeffrey's test
vt.jeffreys_test(test_data_main, 'ratings', 'default_flag', 'predicted_pd', 0.05)

In [ ]:
# Brier score
vt.brier_score(test_data_main, "ratings", "default_flag", "predicted_pd")

In [ ]:
# Hosmer-Lemeshow test
vt.hosmer_test(test_data_main, "ratings", "default_flag", "predicted_pd")